# 02 RAG | 01 Chat Completion | 03 Chat Completion Image

## Azure Environment

## Azure Environment

To execute the sample code Azure service specific information like endpoint, api key etc. is needed ([Details and instructions can be found here](../01_CreateEnvironment/01_Environment.ipynb))

## Step 1: Create OpenAIClient

The `AzureOpenAIClient` from the Azure.AI.OpenAI NuGet package offers a unified interface to create various specialized clients, each designed to handle specific tasks.
In this notebook the `ChatClient` is used.

The multi modal capabilities of the GPT-4o LLM deployment are shown by providing the following photo realistic image together with the user message 'In which style is the provided image? Is it Cartoon or Photorealistic?' the LLM:

![PhotoRealisticImage](../../Assets/Image/SuperBowl_Photorealistic.png)

The image was created using the [04_CreateImageCompletion notebook](./04_CreateImageCompletion.ipynb) and an instance of dall-e-3. 


In [1]:
#r "nuget: Azure.AI.OpenAI, 2.0.0-beta.2"
#r "nuget: DotNetEnv, 2.5.0"

using Azure; 
using Azure.AI.OpenAI;
using OpenAI.Chat;
using DotNetEnv;
using System.ClientModel; 

//configuration file is created during environment creation
static string configurationFile = @"../../Configuration/application.env";
Env.Load(configurationFile);

string oAiApiKey = Environment.GetEnvironmentVariable("WS_AOAI_APIKEY") ?? "WS_AOAI_APIKEY not found";
string oAiEndpoint = Environment.GetEnvironmentVariable("WS_AOAI_ENDPOINT") ?? "WS_AOAI_ENDPOINT not found";
string chatCompletionDeploymentName = Environment.GetEnvironmentVariable("WS_CHATCOMPLETION_DEPLOYMENTNAME") ?? "WS_CHATCOMPLETION_DEPLOYMENTNAME not found";

ApiKeyCredential apiKeyCredential = new ApiKeyCredential(oAiApiKey);
AzureOpenAIClient azureOpenAIClient = new AzureOpenAIClient(new Uri(oAiEndpoint), apiKeyCredential);
ChatClient chatClient = azureOpenAIClient.GetChatClient(chatCompletionDeploymentName);

Console.WriteLine($"AzureOpenAIClient created...");
Console.WriteLine($"ChatClient created...");

Installed Packages Azure.AI.OpenAI, 2.0.0-beta.2 DotNetEnv, 2.5.0

AzureOpenAIClient created...
ChatClient created...


## Step 1: Create Chat Messages

The following cell, demonstrate a basic interaction using the `chatClient` object. In this case, the system message is used to provide instructions or settings for the assistant. It instructs the model to *'Describe the style of images!'*.

The methods `CreateImageMessageContentPart()` and `CreateTextMessageContentPart` of the static object `ChatMessageContentPart` are used to create a n new UserChatMessage instance which can be added to the chat messages collection provided within the model call.

In [2]:
using System.IO;

string systemMessage = "You describe the style of images!";
string userMessage = "In which style is the provided image? Is it Cartoon or Photorealistic?";
string imageFileName = "../../Assets/Image/SuperBowl_Photorealistic.png"; 

ChatMessageContentPart imageMessageContentPart = null;
List<ChatMessage> chatMessages = new List<ChatMessage>();
chatMessages.Add(new SystemChatMessage(systemMessage));

FileStream fileStream = File.OpenRead(imageFileName); 
BinaryData binaryData = BinaryData.FromStream(fileStream);

chatMessages.Add(
    new UserChatMessage(
        ChatMessageContentPart.CreateImageMessageContentPart(
            binaryData,
            "image/png",
            ImageChatMessageContentPartDetail.High
        ), 
        ChatMessageContentPart.CreateTextMessageContentPart(userMessage)
    )
);

Console.WriteLine($"ChatMessages created...");

ChatMessages created...


## Step 2: Call Model

The `ChatCompletionsOptions` object allows a more fine granular control how the model should process the provided chat message collection.

Just like with [text chat completion](../02_01_ChatCompletion/02_ChatCompletion_SDK.ipynb) the method `CompleteChatAsync()` of the `ChatClient` can be used to process the provided image.

In [3]:
ChatCompletionOptions chatCompletionOptions = new ChatCompletionOptions(){
    MaxTokens = 500,
    Temperature = 0.0f,
    TopP = 1.0f,
    FrequencyPenalty = 0.7f,
    PresencePenalty = 0.7f,
};
chatCompletionOptions.StopSequences.Add("\n");

ChatCompletion chatCompletion = await chatClient.CompleteChatAsync(
    messages: chatMessages, 
    options: chatCompletionOptions
);

foreach (ChatMessageContentPart chatMessageContentPart in chatCompletion.Content){
    Console.WriteLine($"Model response: \n {chatMessageContentPart}"); 
}

Model response: 
 The provided image is in a photorealistic style.
